In [ ]:
import torch.nn as nn
import torchvision
from torchvision.models import resnet50, resnet18, squeezenet1_1
from torchvision.models.resnet import ResNet50_Weights, ResNet18_Weights
from torchvision.models import efficientnet_v2_l
from torchvision.models.efficientnet import EfficientNet_V2_L_Weights
from torchvision.models import vgg16, vgg19
from torchvision.models.vgg import VGG16_Weights, VGG19_Weights
from torchvision.models.squeezenet import SqueezeNet1_1_Weights
from torchvision.models import alexnet

import glob
import torch
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
class DEFmodel(nn.Module):
    def __init__(self):
        super(DEFmodel, self).__init__()
        #self.pretrained = squeezenet1_1(weights = SqueezeNet1_1_Weights.DEFAULT)
        #self.pretrained = resnet50(weights = ResNet50_Weights.DEFAULT).eval()
        #self.pretrained = vgg16(weights = VGG16_Weights.DEFAULT)
        #self.pretrained = vgg19(weights = VGG19_Weights.DEFAULT)
        #self.pretrained = resnet18(weights = ResNet18_Weights.DEFAULT).eval()
        self.pretrained = efficientnet_v2_l(weights = EfficientNet_V2_L_Weights.DEFAULT).eval()
        self.pretrained.fc = nn.Identity()

    def forward(self, x):
        x = self.pretrained(x)
        return x

In [ ]:
from deepface.commons import functions

input_dir = "/home/disi/Project-IML/query/"
output_dir = "/home/disi/Project-IML/aligned_query/"

for file_name in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, file_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    detection = functions.extract_faces(img = img, enforce_detection=False)
    x, y, w, h = detection[0][1].values()
    aligned_img = img[int(y):int(y+h), int(x):int(x+w)]
    aligned_img = cv2.cvtColor(aligned_img, cv2.COLOR_BGRA2RGB)
    aligned_img = cv2.resize(aligned_img, (160, 160))
    cv2.imwrite(os.path.join(output_dir, file_name), aligned_img)

input_dir = "/home/disi/Project-IML/gallery/"
output_dir = "/home/disi/Project-IML/aligned_gallery/"

for file_name in os.listdir(input_dir):
    img = cv2.imread(os.path.join(input_dir, file_name))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    detection = functions.extract_faces(img = img, enforce_detection=False)
    x, y, w, h = detection[0][1].values()
    aligned_img = img[int(y):int(y+h), int(x):int(x+w)]
    aligned_img = cv2.cvtColor(aligned_img, cv2.COLOR_BGRA2RGB)
    aligned_img = cv2.resize(aligned_img, (160, 160))
    cv2.imwrite(os.path.join(output_dir, file_name), aligned_img)

In [ ]:
import pickle

data_dir = "/home/disi/Project-IML/aligned_gallery/"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gallery_paths = glob.glob(f"{data_dir}*", recursive=True)

model = DEFmodel()
model.to(device)

embeddings = {}

for path in gallery_paths:
    image = cv2.imread(os.path.join(data_dir, path))
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    #image = cv2.normalize(np.asarray(image), None, 0, 1.0, cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    #image = Image.fromarray(image.astype(np.uint8))
    image = torchvision.transforms.ToTensor()(image)
    image =  torchvision.transforms.Resize((160, 160))(image)
    image = torchvision.transforms.Normalize(mean=(0.5, 0.5,0.5), std=(0.5, 0.5, 0.5))(image)

    image = image.unsqueeze(0)
    image = image.to(device)
    embeddings[path] = model(image).detach().cpu().numpy()

# save embeddings to disk
with open("/home/disi/Project-IML/embeddings.pkl", "wb") as f:
    pickle.dump(embeddings, f)

data_dir = "/home/disi/Project-IML/aligned_query/"
queries = {}
query_paths = glob.glob(f"{data_dir}*", recursive=True)
for query in query_paths:
    image = cv2.imread(os.path.join(data_dir, query))
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2RGB)
    #image = cv2.normalize(np.asarray(image), None, 0, 1.0, cv2.NORM_MINMAX, dtype = cv2.CV_32F)
    #image = Image.fromarray(image.astype(np.uint8))
    image = torchvision.transforms.ToTensor()(image)
    image =  torchvision.transforms.Resize((160, 160))(image)
    image = torchvision.transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(image)
    image = image.unsqueeze(0)
    image = image.to(device)
    queries[query] = model(image).detach().cpu().numpy()

In [ ]:
def euclidian_distance(x, y):
    return np.sqrt(np.sum((x - y)**2))


for query in queries:
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])
    print(f"Query: {query}")
    print(sorted_distances[:5])

# now again but with another distance metric

from sklearn.metrics.pairwise import cosine_similarity

for query in queries:
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = cosine_similarity(query_embedding, gallery_embedding)[0][0]
    sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)
    print(f"Query: {query}")
    print(sorted_distances[:5])

In [ ]:
# iterate over the queries
# set the number of retrieved images to display
num_retrievals = 5

# iterate over the queries
for query in queries:
    # get the query embedding and calculate distances
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = cosine_similarity(query_embedding, gallery_embedding)[0][0]

    # sort the distances in descending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1], reverse=True)

    # get the top retrieved image paths and distances
    top_paths = [x[0] for x in sorted_distances[:num_retrievals]]
    top_similarities = [x[1]*100 for x in sorted_distances[:num_retrievals]]

    # load the query image and the top retrieved images
    images = [Image.open(query)] + [Image.open(path) for path in top_paths]

    # create a figure with subplots for each image
    fig, axes = plt.subplots(1, num_retrievals+1, figsize=(15, 5))

    # display the query image
    axes[0].imshow(np.array(images[0]))
    axes[0].set_title("Query")

    # display the retrieved images and their distances
    for i in range(num_retrievals):
        axes[i+1].imshow(np.array(images[i+1]))
        axes[i+1].set_title(f"Similarity: {top_similarities[i]:.2f}%")
    plt.show()


# set the number of retrieved images to display

# iterate over the queries
for query in queries:
    # get the query embedding and calculate distances
    query_embedding = queries[query]
    distances = {}
    for gallery in embeddings:
        gallery_embedding = embeddings[gallery]
        distances[gallery] = euclidian_distance(query_embedding, gallery_embedding)

    # sort the distances in ascending order
    sorted_distances = sorted(distances.items(), key=lambda x: x[1])

    # get the top retrieved image paths and distances
    top_paths = [x[0] for x in sorted_distances[:num_retrievals]]
    top_distances = [x[1] for x in sorted_distances[:num_retrievals]]

    # load the query image and the top retrieved images
    images = [Image.open(query)] + [Image.open(path) for path in top_paths]

    # create a figure with subplots for each image
    fig, axes = plt.subplots(1, num_retrievals+1, figsize=(15, 5))

    # display the query image
    axes[0].imshow(np.array(images[0]))
    axes[0].set_title("Query")

    # display the retrieved images and their similarities
    for i in range(num_retrievals):
        axes[i+1].imshow(np.array(images[i+1]))
        axes[i+1].set_title(f"EuclidianD: {top_distances[i]:.2f}%")
    plt.show()

The cosine_similarity function returns a matrix of pairwise cosine similarities between the query embedding and the gallery embeddings. Since we are comparing one query image with multiple gallery images, we need to take the first element of the first dimension to get a single similarity score. Also, note that we pass reverse=True to sorted function to sort the distances in descending order since cosine similarity values range between -1 and 1, with higher values indicating more similarity.